In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
import math

In [18]:
FHS_data = pd.read_excel('Framingham_data.xlsx')
FHS_data.head()

,sex,sbp,dbp,scl,chdfate,followup,age,bmi,month,id
0,1,120,80,267.0,1,18,55,25.000000,8,2642
1,1,130,78,192.0,1,35,53,28.400000,12,4627
2,1,144,90,207.0,1,109,61,25.100000,8,2568
3,1,92,66,231.0,1,147,48,26.200001,11,4192
4,1,162,98,271.0,1,169,39,28.400000,11,3977


In [19]:
def proportion_confidence_interval(col: str, CHD: int, gender: int, confidence=0.95):
    data = pd.DataFrame(columns=[col, 'chdfate'])
    data[[col, 'chdfate']] = FHS_data[[col, 'chdfate']].where(FHS_data['chdfate'] == CHD).dropna()
    filtered_data = data.where(data['sex'] == gender)[col].dropna()
    
    n = len(FHS_data['sex'].where(FHS_data['sex'] == gender).dropna())
    print(f'Total recorded datapoints for {col} = {"male" if gender is 1 else "female"} (n): {n}')
    proportion = len(filtered_data.index) / n
    q = 1 - proportion
    print(f'p = {proportion}, q = {q}')
    # Standard error calculated via sigma / sqrt(n)
    std_dev = np.sqrt((proportion * q) / n)
#     print(data.sem())
    print(f'Standard deviation of proportion: {std_dev}')
    z_score = stat.norm.ppf(((1 + confidence) / 2))
#     print(stat.norm.ppf(.975))
    print(f'z-score: {z_score}')
    lower = proportion - (std_dev * z_score)
    upper = proportion + (std_dev * z_score)
    return lower, proportion, upper

In [20]:
def mean_confidence_interval(col: str, CHD: int, confidence=0.95):
    data = pd.DataFrame(columns=[col, 'chdfate'])
    data[[col, 'chdfate']] = FHS_data[[col, 'chdfate']]
    filtered_data = data.where(data['chdfate'] == CHD)[col].dropna()
    
    n = len(data.index)
    print(f'Total recorded datapoints for {col} (n): {n}')
    mean = filtered_data.mean()
    print(f'Mean: {mean}')
    # Standard error calculated via sigma / sqrt(n)
    std_error = filtered_data.std() / np.sqrt(n)
#     print(data.sem())
    print(f'Standard error of the mean: {std_error}')
    t_score = stat.t.ppf(((1 + confidence) / 2), df=(n-1))
#     print(stat.norm.ppf(.975))
    print(f't-score: {t_score}')
    lower = mean - (std_error * t_score)
    upper = mean + (std_error * t_score)
    return lower, mean, upper

In [21]:
def proportion_difference_hypothesis_test(col: str, gender_one: int, gender_two: int, alpha=.05, CHD=1):
    data = pd.DataFrame(columns=[col, 'chdfate'])
    data[[col, 'chdfate']] = FHS_data[[col, 'chdfate']].where(FHS_data['chdfate'] == CHD).dropna()
    filtered_data_one = data.where(data['sex'] == gender_one)[col].dropna()
    filtered_data_two = data.where(data['sex'] == gender_two)[col].dropna()
    
    n_one = len(FHS_data['sex'].where(FHS_data['sex'] == gender_one).dropna())
    n_two = len(FHS_data['sex'].where(FHS_data['sex'] == gender_two).dropna())
    print(f'Total recorded datapoints for {col} = {"male" if gender_one is 1 else "female"} (n_one): {n_one}')
    print(f'Total recorded datapoints for {col} = {"male" if gender_two is 1 else "female"} (n_two): {n_two}')
    p_one = len(filtered_data_one.index) / n_one
    p_two = len(filtered_data_two.index) / n_two
    print(f'p1 = {p_one}, (1 - p1) = {1 - p_one}')
    print(f'p2 = {p_two}, (1 - p2) = {1 - p_two}')
    p_hat = ((n_one * p_one) + (n_two * p_two)) / (n_one + n_two)
    print(f'Combined proportion p-hat: {p_hat}')
#     print(p_one - p_two)
#     print(np.sqrt((p_hat * (1 - p_hat)) * ((1/n_one) + (1/n_two))))
    test_statistic = (p_one - p_two) / np.sqrt((p_hat * (1 - p_hat)) * ((1/n_one) + (1/n_two)))
    print(f'Test statistic (Z): {test_statistic}')
    # Two-tailed hypothesis test, so multiply the area to the left of our test statistic by two to get total area
    p_value = 2 * (1 - stat.norm.cdf(test_statistic))
    print(f'p_value: {p_value}')
    return p_value

In [44]:
def mean_difference_hypothesis_test(col: str, alpha=.05):
    data = pd.DataFrame(columns=[col, 'chdfate'])
    data[[col, 'chdfate']] = FHS_data[[col, 'chdfate']]
    filtered_data_one = data.where(data['chdfate'] == 0)[col].dropna()
    filtered_data_two = data.where(data['chdfate'] == 1)[col].dropna()
    
    n_one = len(filtered_data_one.index)
    n_two = len(filtered_data_two.index)
    print(f'Total recorded datapoints for {col} (no CHD) (n): {n_one}')
    print(f'Total recorded datapoints for {col} (has CHD) (n): {n_two}')
    mean_one = filtered_data_one.mean()
    mean_two = filtered_data_two.mean()
    print(f'Mean_1: {mean_one}')
    print(f'Mean_2: {mean_two}')
    
    var_one = filtered_data_one.var()
    var_two = filtered_data_two.var()
    print(f'Variance of dbp levels (no CHD): {var_one}')
    print(f'Variance of dbp levels (has CHD): {var_two}')
    
    test_statistic = np.abs(mean_one - mean_two) / np.sqrt((var_one / n_one) + (var_two / n_two))
    print(f'Test statistic (T): {test_statistic}')
    # Should change to more general formula for degress of freedom
    dof = math.floor(math.pow((var_one / n_one) + (var_two / n_two), 2) / ((math.pow((var_one / n_one), 2) / (n_one - 1)) + (math.pow((var_two / n_two), 2) / (n_two - 1))))
    print(f'Degrees of Freedom: {dof}')
    # Multiply by two to account for two tailed-ness
    p_value = 2 * (1 - stat.t.cdf(test_statistic, df=dof))
    print(f'P-value: {p_value}')
    return p_value

In [23]:
def evaluate_p_value(p_value):
    if(p_value >= .05):
        print(f'P-value of {p_value} is greater than 0.05. There is insignificant evidence to support the alternative hypothesis')
    else:
        print(f'P-value of {p_value} is less than 0.05. We reject the null hypothesis.')

## Problem 1

### Part A

In [24]:
print("Calculating interval for true US population proportion of women who develop CHD")
interval_women_has_CHD = proportion_confidence_interval(col='sex', CHD=1, gender=2)
print(f'\nWe are 95% confident that the population proportion (proportion of US population) of women diagosed with CHD')
print(f'falls within the interval of {interval_women_has_CHD[0]:.4f} to {interval_women_has_CHD[-1]:.4f}.')

Calculating interval for true US population proportion of women who develop CHD
Total recorded datapoints for sex = female (n): 2650
p = 0.24528301886792453, q = 0.7547169811320755
Standard deviation of proportion: 0.008358009592497936
z-score: 1.959963984540054

We are 95% confident that the population proportion (proportion of US population) of women diagosed with CHD
falls within the interval of 0.2289 to 0.2617.


In [25]:
print("Calculating interval for true US population proportion of men who develop CHD")
interval_men_has_CHD = proportion_confidence_interval(col='sex', CHD=1, gender=1)
print(f'\nWe are 95% confident that the population proportion (proportion of US potppulation) of men diagosed with CHD')
print(f'falls within the interval of {interval_men_has_CHD[0]:.4f} to {interval_men_has_CHD[-1]:.4f}.')

Calculating interval for true US population proportion of men who develop CHD
Total recorded datapoints for sex = male (n): 2049
p = 0.40165934602244996, q = 0.59834065397755
Standard deviation of proportion: 0.010830093725364498
z-score: 1.959963984540054

We are 95% confident that the population proportion (proportion of US potppulation) of men diagosed with CHD
falls within the interval of 0.3804 to 0.4229.


### Part B

$$H_0: P_m - P_f = 0$$ 
$$H_A: \lvert P_m - P_f \rvert \gt 0$$

In [26]:
print("Null hypothesis: There is no difference between population proportions of men who develop CHD and population proportion of women who develop CHD")
print("Alternative hypothesis: There is a significant difference between population proportions of men who develop CHD and women who develop CHD\nAlpha = .05\n")

Null hypothesis: There is no difference between population proportions of men who develop CHD and population proportion of women who develop CHD
Alternative hypothesis: There is a significant difference between population proportions of men who develop CHD and women who develop CHD
Alpha = .05



In [27]:
p_val_male_female_diff = proportion_difference_hypothesis_test(col='sex', gender_one=1, gender_two=2)
evaluate_p_value(p_val_male_female_diff)

Total recorded datapoints for sex = male (n_one): 2049
Total recorded datapoints for sex = female (n_two): 2650
p1 = 0.40165934602244996, (1 - p1) = 0.59834065397755
p2 = 0.24528301886792453, (1 - p2) = 0.7547169811320755
Combined proportion p-hat: 0.31347095126622687
Test statistic (Z): 11.45866683339946
p_value: 0.0
P-value of 0.0 is less than 0.05. We reject the null hypothesis.


## Problem 2

### Part A

In [28]:
interval_DBP_no_CHD = mean_confidence_interval(col='dbp', CHD=0)
print(f'\nWe are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals NOT diagnosed with CHD')
print(f'falls within the interval of {interval_DBP_no_CHD[0]:.2f} mmHg to {interval_DBP_no_CHD[-1]:.2f} mmHg.')

Total recorded datapoints for dbp (n): 4699
Mean: 81.03099814011159
Standard error of the mean: 0.17840415335534265
t-score: 1.9604690658796426

We are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals NOT diagnosed with CHD
falls within the interval of 80.68 mmHg to 81.38 mmHg.


In [29]:
interval_DBP_CHD = mean_confidence_interval(col='dbp', CHD=1)
print(f'\nWe are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals diagnosed with CHD')
print(f'falls within the interval of {interval_DBP_CHD[0]:.2f} mmHg to {interval_DBP_CHD[-1]:.2f} mmHg.')

Total recorded datapoints for dbp (n): 4699
Mean: 85.8499660556687
Standard error of the mean: 0.19252644536384675
t-score: 1.9604690658796426

We are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals diagnosed with CHD
falls within the interval of 85.47 mmHg to 86.23 mmHg.


### Part B

$$H_0: \mu_1 - \mu_2 = 0$$
$$H_A: \lvert \mu_1 - \mu_2 \rvert \gt 0$$

In [45]:
p_val_dbp_diff = mean_difference_hypothesis_test(col='dbp')
evaluate_p_value(p_val_dbp_diff)

Total recorded datapoints for dbp (no CHD) (n): 3226
Total recorded datapoints for dbp (has CHD) (n): 1473
Mean_1: 81.03099814011159
Mean_2: 85.8499660556687
Variance of dbp levels (no CHD): 149.55996904991767
Variance of dbp levels (has CHD): 174.17516474069538
Test statistic (T): 11.877668654501761
Degrees of Freedom: 2665
P-value: 0.0
P-value of 0.0 is less than 0.05. We reject the null hypothesis.


## Problem 3

### Part A

In [31]:
interval_SCL_no_CHD = mean_confidence_interval(col='scl', CHD=0)
print(f'\nWe are 95% confident that the population mean of systolic blood pressure (scl) for individuals NOT diagnosed with CHD')
print(f'falls within the interval of {interval_SCL_no_CHD[0]:.2f} mmHg to {interval_SCL_no_CHD[-1]:.2f} mmHg.')

Total recorded datapoints for scl (n): 4699
Mean: 223.0009375
Standard error of the mean: 0.618649016831971
t-score: 1.9604690658796426

We are 95% confident that the population mean of systolic blood pressure (scl) for individuals NOT diagnosed with CHD
falls within the interval of 221.79 mmHg to 224.21 mmHg.


In [36]:
interval_SCL_has_CHD = mean_confidence_interval(col='scl', CHD=1)
print(f'\nWe are 95% confident that the population mean of systolic blood pressure (scl) for individuals NOT diagnosed with CHD')
print(f'falls within the interval of {interval_SCL_has_CHD[0]:.2f} mmHg to {interval_SCL_has_CHD[-1]:.2f} mmHg.')

Total recorded datapoints for scl (n): 4699
Mean: 239.8431105047749
Standard error of the mean: 0.6836238714998996
t-score: 1.9604690658796426

We are 95% confident that the population mean of systolic blood pressure (scl) for individuals NOT diagnosed with CHD
falls within the interval of 238.50 mmHg to 241.18 mmHg.


### Part B

$$H_0: \mu_1 - \mu_2 = 0$$
$$H_A: \lvert \mu_1 - \mu_2 \rvert \gt 0$$

In [46]:
p_val_SCL_diff = mean_difference_hypothesis_test(col='scl')
evaluate_p_value(p_val_SCL_diff)

Total recorded datapoints for scl (no CHD) (n): 3200
Total recorded datapoints for scl (has CHD) (n): 1466
Mean_1: 223.0009375
Mean_2: 239.8431105047749
Variance of dbp levels (no CHD): 1798.4323217216452
Variance of dbp levels (has CHD): 2196.038167519518
Test statistic (T): 11.734527156046639
Degrees of Freedom: 2602
P-value: 0.0
P-value of 0.0 is less than 0.05. We reject the null hypothesis.
